## Import Libraries

In [6]:
import cv2
import os
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

## Import Env File

In [ ]:
from dotenv import load_dotenv  

load_dotenv()
datasets_path = os.getenv('DATASET_PATH')
models_path = os.getenv('MODELS_PATH')

## Parameters

In [26]:
classes = ['jogja', 'pekalongan']
image_size = (224, 224) 
test_split_ratio = 0.2

## GLCM Function

In [ ]:
def extract_glcm_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return [contrast, dissimilarity, homogeneity, energy, correlation]

## Load The Dataset and Extract The Features

In [18]:
# features = []
# labels = []
# for class_idx, class_name in enumerate(classes):
#     folder = f'{datasets_path}/{class_name}'
#     for img_name in os.listdir(folder):
#         img_path = os.path.normpath(os.path.join(folder, img_name))
#         if not os.path.exists(img_path):
#             print(f"File not found: {img_path}")
#             continue
#         image = cv2.imread(img_path)
#         if image is None:
#             print(f"Failed to read image: {img_path}")
#             continue
#         if not img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
#             print(f"Unsupported file format: {img_name}")
#             continue
#         print(f"Processing: {img_path}")
#         if image is None:
#             print(f"Image failed to load: {img_path}")
#             continue

In [ ]:
features = []
labels = []
for class_idx, class_name in enumerate(classes):
    folder = f'{datasets_path}/{class_name}'
    for img_name in os.listdir(folder):
        img_path = os.path.normpath(os.path.join(folder, img_name))
        image = cv2.imread(img_path)
        image = cv2.resize(image, image_size)
        glcm_features = extract_glcm_features(image)
        features.append(glcm_features)
        labels.append(class_idx)
        
features = np.array(features)
labels = np.array(labels)

## Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_split_ratio, random_state=42)

In [23]:
# Melihat 10 baris pertama dari X_train
print("X_train (10 baris pertama):")
print(X_train[:10])

# Melihat 10 baris pertama dari X_test
print("\nX_test (10 baris pertama):")
print(X_test[:10])

# Melihat 10 baris pertama dari y_train
print("\ny_train (10 baris pertama):")
print(y_train[:10])

# Melihat 10 baris pertama dari y_test
print("\ny_test (10 baris pertama):")
print(y_test[:10])

X_train (10 baris pertama):
[[9.80751922e+02 1.78484946e+01 1.49872153e-01 1.32093245e-02
  8.67884478e-01]
 [1.60364778e+03 2.39249680e+01 8.93015971e-02 1.54187560e-02
  8.93606726e-01]
 [6.29888773e+02 1.28200673e+01 3.12479088e-01 4.15703108e-02
  9.10122010e-01]
 [5.90234549e+03 5.15063661e+01 7.20586887e-02 1.05535844e-02
  5.28418849e-01]
 [1.08823469e+03 1.56340687e+01 4.26539764e-01 1.53007216e-01
  9.21310516e-01]
 [1.24647732e+03 2.28366232e+01 1.46619304e-01 1.59562402e-02
  6.78360870e-01]
 [2.20761703e+03 2.42389894e+01 3.48166760e-01 5.57208919e-02
  8.32559692e-01]
 [1.39260558e+03 2.22038357e+01 2.10868107e-01 4.60701592e-02
  8.47746935e-01]
 [3.23252751e+03 3.84462284e+01 9.46308530e-02 2.32261934e-02
  7.52383186e-01]
 [1.09290347e+03 2.28722373e+01 8.36116801e-02 1.11353112e-02
  6.19714869e-01]]

X_test (10 baris pertama):
[[6.61612608e+02 1.53084761e+01 2.15124548e-01 1.79455517e-02
  9.37382133e-01]
 [3.34264110e+03 3.93606262e+01 4.87802995e-02 9.24727551e-03
 

## Train SVM

In [20]:
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

## Evaluates

In [21]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 52.08%


## Save The Model

In [22]:
from joblib import dump

# Simpan model ke file
dump(clf, models_path+'/svm_glcm_model.joblib')
print("Model berhasil disimpan ke svm_glcm_model.joblib")

Model berhasil disimpan ke svm_glcm_model.joblib
